# Lab 19

Today, we will dig a bit more into evaluating the efficacy of our code via _benchmarking._ Today'a goals are: 

0. Define different kinds of benchmarking 
1. Deploy different kinds of benchmarking

## Machine Learning at Scale

Part of working on large data and complex code is to explore how long each piece of code is taking. Today, we will return to our implementation of gradient descent and use that as our example. To prepare for that, please import the data from Lab 9 and add to the below functions (from Lab 11):

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import numpy as np
from numpy import linalg as LA
import pandas as pd

import timeit ## <-- New line!
import time

# import line_profiler
# import memory_profiler

from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Functions for later use 
    
def compute_mse(truth_vec, predict_vec):
    return np.mean((truth_vec - predict_vec)**2)
    
def compute_m_partial(in_vals, truth_vec, predict_vec):
    return -2*np.mean(in_vals*(truth_vec-predict_vec))

def compute_b_partial(truth_vec, predict_vec):
    return -2*np.mean(truth_vec-predict_vec)

def grad_des(input_data, truth_vec, max_steps):
    # Add your implementation for gradient descent here
    pass

def minibatch_gd(input_data, truth_vec, batch_size, max_steps):
    # Add your implementation for mini-batch gradient descent here
    pass

def stochastic_gd(input_data, truth_vec, max_steps):
    # Add your implementation for stochastic gradient descent here
    pass

#### Data

This time, we will use both the data from Labs 9 and 16. Please add a copy of each to this directory. 

In [3]:
## Import Data

employ_data = pd.read_csv("lab9data.csv", sep = ",")

## numpy vectors of our inputs
neuro = employ_data[["neuroticism"]].to_numpy()
perform = employ_data[["performance"]].to_numpy()

In [16]:
## Import Weather Data

weather_data = np.genfromtxt("lab16data.csv", delimiter=',', skip_header=1)
weather_pd = pd.read_csv("lab16data.csv", sep = ",")

# Split into the input variables and the target classes
in_weather = weather_data[:,:4]
out_class = weather_data[:,4]

# Get the variable names 
var_names = list(weather_pd.columns)[:4]

OSError: lab16data.csv not found.

## Timing our implementations with `timeit`

Last time, we used the `time` module to time how long it takes an implementation to run. Another option is to use **magic** built in python commands to check individual lines. These commands take the form of `%command` for a single line and `%%command` for a block of code. Today, we will use a few of them starting with `%%timeit`.

Noting that there can be small changes in run time, in `timeit`, we run the code several times to find the average run time. Notice how the output between `%%time` and `%%timeit` are different: 

In [5]:
%%timeit

# Specify and fit the model
grove = RandomForestClassifier(n_estimators=10, max_features = 3, max_depth=2, random_state=0)
grove.fit(in_weather, out_class)


12.6 ms ± 411 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%time

# Specify and fit the model
grove = RandomForestClassifier(n_estimators=10, max_features = 3, max_depth=2, random_state=0)
grove.fit(in_weather, out_class)

CPU times: user 17.1 ms, sys: 1.79 ms, total: 18.9 ms
Wall time: 19.8 ms


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

##### What do you notice?

Did one of the blocks take longer than the other one? Why do you think this is?  

**Your thoughts**

Whare are two things that you notice that are similar about these outputs?

1. 
2. 

What are two things that are different about these outputs? 

1. 
2. 

## Benchmarking

Last time (and above), we looked at how long a whole algorithm would take to do its work. This is a good first step, but when we _benchmark_ code, we examine how fast each piece of the code is. This means that we need a method that will tell us how each piece runs. We could do this by running a time line for each individual piece of our code, OR we could use a _profiler_ which gives us more nuanced information about the run times without us having to insert additional timing lines. 

Benchmarking as a whole allows us to determine if we need to edit or change any lines due to _bottlenecking_ (or places where the code slows down).   

As a visual, consider a bottle like a glass soda bottle with a long neck. Suppose you filled the bottle with little balls, and then turned the bottle upside down quickly. Would all the balls fall out at once or would there be a sort of traffic jam at one place where the balls would have to wait before falling the rest of the way? This place where they are waiting is the _bottleneck,_ hence the term "bottlenecking."

Today, we will look at three kinds of _profilers._ Specifically, we are looking at:
0. Profiling a whole script
1. Line by line Profiling
2. Memory Profiling 

For this part, we need to conda install a few things:
* `line_profiler`
* `memory_profiler`

The remainder of this lab follows this [blog post](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html).

### (First) Profiler

The first _magic_ line that we will use is `%prun` which is the profiler command. This is akin to `cProfile` or `profile` in usual [profilers for python scripting](https://docs.python.org/3/library/profile.html). This will tell you every piece (including deep parts of the base python) that are touched by your code. 

This profiler will give us [various timing information](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-prun). Run the below line and consult the linked helpfile to see what the profiler is telling you. 

In [7]:
%prun grove.fit(in_weather, out_class)

#### What does it all mean? 

This is a lot of information. `%prun` gives us **every** piece and **every** line (even those in deep, deep python). This output can be super overwhelming. 

Let's look at a few pieces:   
* First, let's look at the column names (as described [here](https://stackoverflow.com/questions/7069733/how-do-i-read-the-output-of-the-ipython-prun-profiler-command): 
  - `ncalls` is the number of classes to that line. 
  - `tottime` is the total running time per call (without including time calling helper functions)
  - `cumtime` is the total running time per call (including time calling helper functions)
* What gets called a lot? What gets called less? (Check out the `ncalls` column)
* Next isolate the lines are taking up a lot of time in the `tottime` column 
* Now isolate the lines are taking up a lot of time in the `cumtime` column

Are these all the same lines? Does this surprise you? 

#### Your turn

Run `%prun` on your implementation of `grad_des` and on `minibatch_gd`. 

In [8]:
# Run %prun on grad_des

Which lines are taking the most time? 

**Your thoughts** 

Which lines are called the most? 

**Your thoughts** 

What surprises you about this analysis? 

**Your thoughts** 

In [9]:
# Run %prun on minibatch_gd

Which lines are taking the most time? 

**Your thoughts** 

Which lines are called the most? 

**Your thoughts** 

What surprises you about this analysis? 

**Your thoughts** 


### Line Profiler

Instead of looking at a full script and every python code that it calls, we might want to look how much time each line (just in our program) takes to execute. This will tells us how long each line we coded takes to run. 

To start this process, we will must load in the `line_profiler` as "magic" functions

In [10]:
%load_ext line_profiler

Once loaded as "magic" we can use `%lprun` which will run our functions, timing them line by line. Let's do a silly example. Before running the line profiler, which line do you think will take longer: setting up the `grove` (line 1 of `tune_fit`) or fitting grove (line 2 of `tune_fit`)?

**Your Thoughts**

In [11]:
def tune_fit(in_vars,classes):
    grove = RandomForestClassifier(n_estimators=10, max_features = 3, max_depth=2, random_state=0)
    grove.fit(in_vars,classes)

In [17]:
%lprun -f tune_fit tune_fit(in_weather, out_class)

#### What does it all mean? 

The output of `%lprun` gives us a line by line timing (including number of times called). Notice that the code is not running for over 30K seconds, rather that the `Timer unit` is 0.00000001 of a second (or `1e-06`).


#### Your turn

Run `%lprun` on your implementation of `grad_des` and on `minibatch_gd`. What is surprising about these two functions? 

In [ ]:
# Run %lprun on grad_des

Which lines are taking the most time? 

**Your thoughts** 

Which lines are called the most? 

**Your thoughts** 

What surprises you about this analysis? 

**Your thoughts** 

In [ ]:
# Run %lprun on minibatch_gd

Which lines are taking the most time? 

**Your thoughts** 

Which lines are called the most? 

**Your thoughts** 

What surprises you about this analysis? 

**Your thoughts** 

### Memory Profiler

Instead of looking at timing of each line, we might want to look how much memory each line takes to execute as well as the total memory for the function. 

To start this process, we will must load in the `memory_profiler` as "magic" functions

In [14]:
%load_ext memory_profiler

Now that it is loaded as "magic", we can use two functions `%memit` which gives use the total memory usage and `%mprun` which gives us a line by line assessment of memory. 


In [15]:
%memit tune_fit(in_weather, out_class)

peak memory: 122.23 MiB, increment: 0.02 MiB


#### Your turn

Run `%memit` on your implementation of `grad_des` and on `minibatch_gd`. What is surprising about these two functions? 

In [ ]:
# Run %memit on grad_des

In [ ]:
# Run %memit on minibatch_gd

Compare your results from above. What was your peak memory for each? Which one uses memory in a more efficient manner? 

**Your thoughts**

(If you would like to see a line by line readout for memory usage, check out `mprun` below the final thoughts section.)

## Why do we care?

In this lab, we've used very small examples. But as we scale up our data, it is important to pay attention to what lines use more memory and take more time. These bottlenecks can make or break our machine learning implementations, and resolving them are critical to scaling up implementations to use on larger and larger datasets. 

### Final Thoughts

To finish up this lab, answer the question: **what did you learn by benchmarking two versions of gradient descent?** Share your thoughts in a post on **#lab_submission** channel on slack with your answer. 

If your have questions from this lab, post them to #lab_questions with the same preamble (i.e. starting with **Lab19**). If you have the same question, please use one of the emoji's to upvote the question. If you would like to answer someone's question, please use the thread function. This will tie your answer to their question. 

### Next Time

We will start our journey into deep learning. 

#### Bonus - `mprun` 

Like `prun` and `lprun`, `mprun` does a line by line read out on memory usage. 

To run `mprun`, we do have to create a file for our example before we can run it. Again following the [earlier blog post](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html):

In [ ]:
%%file mprun_demo.py

from sklearn.ensemble import RandomForestClassifier
import numpy as np
from numpy import linalg as LA
import pandas as pd

def tune_fit(in_vars,classes):
    grove = RandomForestClassifier(n_estimators=10, max_features = 3, max_depth=2, random_state=0)
    grove.fit(in_vars,classes)

Let's again turn to our a silly example. Before running the memory profiler that we just created, which line do you think will take more memory? Setting up the `grove` (line 1 of `tune_fit`) or fitting grove (line 2 of `tune_fit`)

**Your thoughts** 

In [ ]:
from mprun_demo import tune_fit
%mprun -f tune_fit tune_fit(in_weather, out_class)

#### Resources consulted 

0. [Benchmarking your code](https://rbspy.github.io/benchmarking-your-code/)
1. [IPython Magic Commands](https://jakevdp.github.io/PythonDataScienceHandbook/01.03-magic-commands.html)
2. [Profiling and Timing Code](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html)
3. [How do I get time of a Python program's execution?](https://stackoverflow.com/questions/1557571/how-do-i-get-time-of-a-python-programs-execution)
4. [Measure Time in Python – time.time() vs time.clock()](https://www.pythoncentral.io/measure-time-in-python-time-time-vs-time-clock/#:~:text=The%20first%20type%20of%20time,called%20elapsed%20or%20running%20time.)  
5. I also googled 1e-6 to check for the correct number of 0's after the decimal